# Code for the topic modelling

Tuto - https://www.youtube.com/watch?v=arMTVXEbV9Q

## Import packages

In [4]:
# nltk
import nltk
# gensim
import gensim
import gensim.corpora as corpora
# spacy
import spacy
#vis
import pyLDAvis
import pyLDAvis.gensim

In [17]:
import pandas as pd

## Import data

In [ ]:
db_init = pd.read_csv("C:/Users/valentin.stuhlfauth/OneDrive - univ-lyon2.fr/2_2025-DUT systematic review/2_Topic modelling/0_code/2025_Pol4PED_systematic_review/2025-02-27_Screening_result.csv",sep=';',encoding='ISO-8859-1')

In [14]:
db_init["Approach"].unique()

array(['Conceptual', 'Methodological', 'Qualitative', nan, 'Reviews',
       'Quantitative', 'Modelling', 'Ingineering'], dtype=object)

In [72]:
db_init=db_init.reset_index()

In [42]:
df_text = pd.DataFrame()
df_text["text_topic"] = data["Article Title"] + " " + data["Abstract"]
df_text = df_text["text_topic"]

## Pretreatment

### Lemmatisation, generation and bigrams

In [44]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [46]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

In [47]:
data_words = gen_words(lemmatization(df_text))

In [93]:
#BIGRAMS AND TRIGRAMS
## threshold (float, optional) – Represent a score threshold for forming the phrases (higher means fewer phrases) (init = 100)
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=10)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=10)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0])

['theory', 'paper_analyze', 'relationship', 'low_carbon_city', 'climate_change', 'background', 'global_warming', 'low_carbon_city', 'construction', 'important', 'measure', 'mitigate_climate_change', 'crisis', 'low_carbon_urban', 'planning', 'key', 'technology', 'low_carbon_urban', 'construction', 'research', 'aspect', 'low_carbon_urban', 'planning', 'theory', 'concept', 'practice', 'low', 'zero', 'carbon', 'community', 'low_carbon_city', 'development', 'future', 'sum', 'low_carbon_urban', 'planning', 'research', 'progress', 'domestic', 'overseas', 'put', 'future', 'trend', 'low_carbon_urban', 'planning', 'research', 'aspect', 'demarcation', 'path', 'low_carbon_city', 'low_carbon_urban', 'planning', 'construction', 'popularity', 'low_carbon_technology', 'enforcement', 'mechanism', 'innovation', 'low_carbon_city', 'construction']


### TF-IDF (play with hypervalue)

In [94]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.02
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [95]:
list(dict.fromkeys(words))

['use',
 'urban',
 'base',
 'low_carbon',
 'impact',
 'level',
 'city',
 'study',
 'development',
 'policy',
 'model',
 'analysis',
 'paper',
 'promote',
 'carbon_emission',
 'energy',
 'high',
 'reduce',
 'effect',
 'research',
 'challenge',
 'focus',
 'provide',
 'method',
 'also',
 'find',
 'build',
 'potential',
 'role',
 'affect',
 'approach',
 'goal',
 'need',
 'improve',
 'low_carbon_city',
 'datum',
 'different',
 'result',
 'increase',
 'area',
 'system',
 'low',
 'show',
 'such',
 'carbon',
 'develop',
 'more',
 'important',
 'sustainable',
 'large',
 'design',
 'local',
 'contribute',
 'include',
 'however',
 'apply',
 'emission',
 'consider',
 'construction',
 'analyze',
 'economic',
 'green',
 'strategy',
 'explore',
 'support',
 'evidence',
 'achieve',
 'implementation',
 'result_show',
 'take',
 'evaluate',
 'community',
 'concept',
 'innovation',
 'finally',
 'finding',
 'term',
 'transition',
 'comprehensive',
 'reveal',
 'decrease',
 'value',
 'measure',
 'environment

### Manual Thesaurus

In [84]:
# Stop words 
stop_words=[]
stop_words.append(['CO2'])

print(stop_words)

[['CO2']]


In [85]:
# Stop words suppression 
data_bigrams_trigrams_stop = [[mot for mot in doc if not mot in stop_words] for doc in data_bigrams_trigrams]

## Topic modelling global

### Dictionary

In [96]:
from gensim.corpora import Dictionary
dico = Dictionary(data_bigrams_trigrams)
print(dico)

Dictionary<8958 unique tokens: ['aspect', 'background', 'carbon', 'climate_change', 'community']...>


In [97]:
id2word = corpora.Dictionary(data_bigrams_trigrams)

corpus = []
for text in data_bigrams_trigrams:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 4), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 5), (19, 1)]
aspect


### LDA (init passes=10)

alpha: Controls the sparsity of the document-topic distribution.
- A small alpha will lead to fewer topics being assigned to each document (sparse document-topic distribution).
- A larger alpha will assign more topics to each document (denser document-topic distribution).

beta (or eta): Controls the sparsity of the topic-term distribution.
- A small beta means that topics will have fewer terms associated with them.
- A large beta allows more terms per topic.

In [136]:
# lissing topic appartenance = alpha (close to 0 = crisp (binary) / more than 10 = lissing of the results) (deffault = 1/num_topics // "auto")
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                           id2word=id2word,
                                           num_topics=30,
                                           #random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=5,
                                           alpha=0.05,
                                           eta= 0.05)

In [129]:
lda_model.print_topics()[0:2]

[(1,
  '0.130*"climate" + 0.066*"traffic" + 0.063*"travel" + 0.060*"decouple" + 0.038*"choice" + 0.037*"public_transport" + 0.036*"currently" + 0.033*"big" + 0.032*"geographical_location" + 0.025*"characteristic"'),
 (9,
  '0.065*"carbon_emission" + 0.042*"city" + 0.029*"effect" + 0.023*"study" + 0.023*"impact" + 0.021*"spatial" + 0.020*"green" + 0.019*"increase" + 0.017*"high" + 0.016*"carbon"')]

In [130]:
lda_model.show_topic(0)

[('sector', 0.064044744),
 ('industrial', 0.060455568),
 ('account', 0.045405537),
 ('co', 0.03962339),
 ('generate', 0.028427742),
 ('global', 0.027625723),
 ('material', 0.02548007),
 ('multi', 0.024007201),
 ('small', 0.023523958),
 ('city', 0.022285305)]

In [131]:
# serach terms in topic
lda_model.get_term_topics("sharing", minimum_probability=0)

[(22, 0.0014820119)]

### Tuning of the parameters

List of hyperparameters to test
- num_topics_list = [5, 10, 15, 25, 30]  # Example list of num_topics values 
- alpha_list = ['symmetric', 'asymmetric', 0.01, 0.05, 0.1]  # Different alpha values 
- beta_list = [0.01, 0.05, 0.1, 'auto']  # Different beta values 

*Results*
Best Coherence: 0.4889965028444934
Best Parameters: num_topics=30, alpha=0.1, beta=0.1

List of hyperparameters to test
- num_topics_list = [25, 30, 35, 40, 45]  # Example list of num_topics values
- alpha_list = [0.1, 0.2, 0.5, 1, 1.5, 2]  # Different alpha values
- beta_list = [0.1, 0.2, 0.5, 1, 1.5, 2]  # Different beta values

Best Coherence: 0.7895028923161037
Best Parameters: num_topics=30, alpha=0.1, beta=2

In [ ]:
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Define a function to compute coherence
def compute_coherence(model, corpus, dictionary, texts):
    coherence_model_lda = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    return coherence_model_lda.get_coherence()

# List of hyperparameters to test
num_topics_list = [25,28, 30,32, 35]  # Example list of num_topics values
alpha_list = [0.05,0.075,0.1, 0.15, 0.2]  # Different alpha values
beta_list = [1.5, 1.8, 2,2.2,2.5,3,4]  # Different beta values

# Initialize variables for tracking the best model and coherence score
best_coherence = -1
best_params = None
best_model = None

# Perform grid search
for num_topics in num_topics_list:
    for alpha in alpha_list:
        for beta in beta_list:
            # Initialize and train the LDA model with current hyperparameters
            lda_model = LdaModel(corpus=corpus[:-1], 
                                 id2word=id2word, 
                                 num_topics=num_topics, 
                                 alpha=alpha, 
                                 eta=beta, 
                                 update_every=1, 
                                 chunksize=100, 
                                 passes=5)
            
            # Calculate the coherence score for this model
            coherence = compute_coherence(lda_model, corpus, id2word, texts)
            
            # Update the best model if we get a higher coherence score
            if coherence > best_coherence:
                best_coherence = coherence
                best_params = (num_topics, alpha, beta)
                best_model = lda_model

# Output the best results
print(f"Best Coherence: {best_coherence}")
print(f"Best Parameters: num_topics={best_params[0]}, alpha={best_params[1]}, beta={best_params[2]}")

### Vizualizing the Data

In [137]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15     0.400302  0.255413       1        1  28.447914
7      0.443909 -0.134930       2        1  18.367795
28     0.456495  0.053488       3        1  12.321923
0      0.399785  0.163204       4        1  11.563895
12     0.481509 -0.066905       5        1   5.585966
24     0.333373 -0.291895       6        1   4.098572
2      0.118817  0.435387       7        1   3.330636
3      0.226100  0.326330       8        1   3.295642
13     0.036262 -0.491000       9        1   2.574056
9     -0.360499  0.299366      10        1   2.240578
25    -0.180731  0.381539      11        1   1.192922
16     0.154428 -0.352954      12        1   1.151224
20    -0.316422 -0.267410      13        1   0.631483
4      0.043496  0.181325      14        1   0.562962
11    -0.103595  0.289274      15        1   0.550968
26    -0.052626 -0.305435      16        1   0.524894
29    -0.183453 -0.239733      17        1   0.438765
21    -0.318315  0.054812      18        1   0.403182
10    -0.327647 -0.089158      19        1   0.365196
6     -0.204764  0.093404      20        1   0.340066
1     -0.088487  0.131856      21        1   0.321485
27    -0.161059 -0.016261      22        1   0.319636
5     -0.233692 -0.042840      23        1   0.316812
8     -0.042607 -0.136789      24        1   0.282023
14     0.021508 -0.050962      25        1   0.274044
18    -0.135650 -0.097489      26        1   0.199438
23    -0.093423 -0.015778      27        1   0.096434
17    -0.103419 -0.023632      28        1   0.073516
19    -0.104900 -0.020799      29        1   0.066188
22    -0.104696 -0.021426      30        1   0.061786, topic_info=              Term         Freq        Total Category  logprob  loglift
102         energy  2135.000000  2135.000000  Default  30.0000  30.0000
93           urban  4980.000000  4980.000000  Default  29.0000  29.0000
177          green  1112.000000  1112.000000  Default  28.0000  28.0000
697       emission  1752.000000  1752.000000  Default  27.0000  27.0000
191         policy  1882.000000  1882.000000  Default  26.0000  26.0000
..             ...          ...          ...      ...      ...      ...
25   paper_analyze     0.014854    54.365868  Topic30  -9.1003  -0.8160
26            path     0.014854   122.398119  Topic30  -9.1003  -1.6275
27        planning     0.014854   829.686602  Topic30  -9.1003  -3.5413
28      popularity     0.014854     4.067368  Topic30  -9.1003   1.7768
29        practice     0.014854   170.304707  Topic30  -9.1003  -1.9578

[1377 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
349      19  0.788191          absence
1111      5  0.973020         absolute
1002     17  0.968929         academic
201      12  0.949973       acceptance
517       1  0.992380           access
...     ...       ...              ...
672      11  0.951739  well_understand
1446     17  0.957078             west
298       1  0.990815             wide
443       1  0.974460            world
443       2  0.021751            world

[1405 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 8, 29, 1, 13, 25, 3, 4, 14, 10, 26, 17, 21, 5, 12, 27, 30, 22, 11, 7, 2, 28, 6, 9, 15, 19, 24, 18, 20, 23])

## Topic modelling "Scale and Resources" > 

## Data analysis

In [64]:
# description of topics' space
doc_topics = lda_model.get_document_topics(corpus[:-1])
print(doc_topics)

In [65]:
# "sparce" matrix transformation
from gensim.matutils import corpus2csc
mat_sparce = corpus2csc(doc_topics)

print(mat_sparce)

  (5, 0)	0.026408571749925613
  (6, 0)	0.017804166302084923
  (10, 0)	0.08102951943874359
  (15, 0)	0.028569821268320084
  (17, 0)	0.014475645497441292
  (21, 0)	0.013427573256194592
  (26, 0)	0.16010437905788422
  (29, 0)	0.018302064388990402
  (32, 0)	0.015604554675519466
  (33, 0)	0.05355236306786537
  (34, 0)	0.05800873786211014
  (39, 0)	0.03783108666539192
  (42, 0)	0.08864591270685196
  (46, 0)	0.19876927137374878
  (47, 0)	0.017536601051688194
  (49, 0)	0.10699625313282013
  (5, 1)	0.04468901455402374
  (6, 1)	0.041244301944971085
  (13, 1)	0.021645143628120422
  (15, 1)	0.023561712354421616
  (18, 1)	0.013892151415348053
  (21, 1)	0.013131385669112206
  (24, 1)	0.04393310099840164
  (26, 1)	0.06432025879621506
  (29, 1)	0.03627924993634224
  :	:
  (20, 2025)	0.015533776953816414
  (24, 2025)	0.011222349479794502
  (27, 2025)	0.014651557430624962
  (29, 2025)	0.12063784152269363
  (33, 2025)	0.20882399380207062
  (34, 2025)	0.023475809022784233
  (38, 2025)	0.02848108299076557


In [66]:
# data frame
mat_dt = mat_sparce.T.toarray()
mat_dt

array([[0.        , 0.        , 0.        , ..., 0.0175366 , 0.        ,
        0.10699625],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.08628862],
       [0.        , 0.        , 0.02039475, ..., 0.        , 0.        ,
        0.20102744],
       ...,
       [0.02747214, 0.01926056, 0.        , ..., 0.01365835, 0.        ,
        0.21855922],
       [0.02381448, 0.        , 0.        , ..., 0.05243335, 0.        ,
        0.14668371],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.15873881]])

In [74]:
# data frame with polarities
dfTopic = pd.DataFrame(mat_dt, columns = ["T" + str(i) for i in range(mat_dt.shape[1])])
df_Topic_treatment = dfTopic
df_Topic_treatment['id'] = db_init['index']

print(df_Topic_treatment.head())

    T0        T1        T2   T3   T4        T5        T6   T7        T8   T9  \
0  0.0  0.000000  0.000000  0.0  0.0  0.026409  0.017804  0.0  0.000000  0.0   
1  0.0  0.000000  0.000000  0.0  0.0  0.044689  0.041244  0.0  0.000000  0.0   
2  0.0  0.000000  0.020395  0.0  0.0  0.028749  0.000000  0.0  0.097484  0.0   
3  0.0  0.000000  0.000000  0.0  0.0  0.010393  0.000000  0.0  0.000000  0.0   
4  0.0  0.067151  0.000000  0.0  0.0  0.000000  0.000000  0.0  0.000000  0.0   

   ...  T41       T42  T43  T44       T45       T46       T47  T48       T49  \
0  ...  0.0  0.088646  0.0  0.0  0.000000  0.198769  0.017537  0.0  0.106996   
1  ...  0.0  0.054272  0.0  0.0  0.017556  0.205746  0.000000  0.0  0.086289   
2  ...  0.0  0.016196  0.0  0.0  0.000000  0.262996  0.000000  0.0  0.201027   
3  ...  0.0  0.015630  0.0  0.0  0.000000  0.135810  0.000000  0.0  0.265583   
4  ...  0.0  0.000000  0.0  0.0  0.000000  0.125462  0.000000  0.0  0.047965   

   id  
0   0  
1   1  
2   2  
3   3 